<a href="https://colab.research.google.com/github/jsedoc/ConceptorDebias/blob/master/EEC_with_CN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget http://www.saifmohammad.com/WebDocs/EEC/Equity-Evaluation-Corpus.zip

--2019-02-27 18:35:55--  http://www.saifmohammad.com/WebDocs/EEC/Equity-Evaluation-Corpus.zip
Resolving www.saifmohammad.com (www.saifmohammad.com)... 192.185.17.122
Connecting to www.saifmohammad.com (www.saifmohammad.com)|192.185.17.122|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102568 (100K) [application/zip]
Saving to: ‘Equity-Evaluation-Corpus.zip’

Equity-Evaluation-C 100%[===================>] 100.16K   548KB/s    in 0.2s    

2019-02-27 18:36:00 (548 KB/s) - ‘Equity-Evaluation-Corpus.zip’ saved [102568/102568]



In [0]:
!unzip Equity-Evaluation-Corpus.zip

Archive:  Equity-Evaluation-Corpus.zip
  inflating: Equity-Evaluation-Corpus.csv  
  inflating: README.txt              


In [0]:
import numpy as np
import scipy, requests, codecs, os, re, nltk, itertools, csv
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering, KMeans
import tensorflow as tf
from scipy.stats import spearmanr
import pandas as pd
import functools as ft
import os
import io
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
EEC = pd.read_csv('/content/Equity-Evaluation-Corpus.csv', header=0)

In [0]:
EEC[0:5]


,ID,Sentence,Template,Person,Gender,Race,Emotion,Emotion word
0,2018-En-mystery-05498,Alonzo feels angry.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,angry
1,2018-En-mystery-11722,Alonzo feels furious.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,furious
2,2018-En-mystery-11364,Alonzo feels irritated.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,irritated
3,2018-En-mystery-14320,Alonzo feels enraged.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,enraged
4,2018-En-mystery-14114,Alonzo feels annoyed.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,annoyed


# Load GloVe embeddings

In [0]:
!pip install -q gdown
!gdown https://drive.google.com/uc?id=1Ty2exMyi-XOufY-v81RJfiPvnintHuy2

Downloading...
From: https://drive.google.com/uc?id=1Ty2exMyi-XOufY-v81RJfiPvnintHuy2
To: /content/gensim_glove.840B.300d.txt.bin
2.65GB [00:49, 53.8MB/s]


# Load Word2vec embeddings
CN performed better using word2vec embeddings. For all 4 sentiments, CN showed improvements on debiasing gender bias.
If using GloVe embeddings, CN showed improvements on anger, fear and joy. For the sentiment 'sadness', CN gave a higher gender difference. However, by my observations, results can be improved if using more female and male names to find conceptors. So with a larger name datasets, the results of 'sadness' may also show improvements on debiasing.

In [0]:
!gdown https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM

Downloading...
From: https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM
To: /content/GoogleNews-vectors-negative300.bin.gz
1.65GB [00:10, 154MB/s]


In [0]:
!gunzip /content/GoogleNews-vectors-negative300.bin.gz

# Load male and female name lists

In [0]:
!wget https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/names/male.txt
!wget https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/names/female.txt

--2019-02-27 19:32:08--  https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/names/male.txt
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20466 (20K) [text/plain]
Saving to: ‘male.txt’

male.txt            100%[===================>]  19.99K  --.-KB/s    in 0.04s   

2019-02-27 19:32:08 (491 KB/s) - ‘male.txt’ saved [20466/20466]

--2019-02-27 19:32:09--  https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/names/female.txt
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35751 (35K) [text/plain]
Saving to: ‘female.txt’

female.txt          100%[===================>]  34.91K  --.-KB/s    in 0.08s   

2019-02-27 19:32:09 (460 KB/s) - ‘female.txt’ saved [35751/35751]



In [0]:
import gensim

from gensim.models.keyedvectors import KeyedVectors


In [0]:
glove = KeyedVectors.load_word2vec_format('/content/' + 'gensim_glove.840B.300d.txt.bin', binary=True)

In [0]:
word2vec = KeyedVectors.load_word2vec_format('/content/' + 'GoogleNews-vectors-negative300.bin', binary=True)

# Get frequency list (used in calculating sentence embeddings)

In [0]:
!git clone https://github.com/PrincetonML/SIF

Cloning into 'SIF'...
remote: Enumerating objects: 128, done.
remote: Total 128 (delta 0), reused 0 (delta 0), pack-reused 128
Receiving objects: 100% (128/128), 2.80 MiB | 6.65 MiB/s, done.
Resolving deltas: 100% (55/55), done.


In [0]:
wikiWordsPath = '/content' + '/SIF/auxiliary_data/enwiki_vocab_min200.txt' # https://github.com/PrincetonML/SIF/blob/master/auxiliary_data/enwiki_vocab_min200.txt

frequencies = {}
with open(wikiWordsPath, "r+") as f_in:
    for line in f_in:
        frequencies[line.split(' ')[0]] = float(line.split(' ')[1])

# Load gender names

In [0]:
#load names from larger name dataset
f = open('/content/male.txt', 'r+')
i = 0
word_list= []
for line in f:
  i+=1
  if i < 7:
    continue
  token = line.split()
  word_list.append(token[0])
f.close()

In [0]:
#load names from larger name dataset
f = open('/content/female.txt', 'r+')
i = 0
for line in f:
  i+=1
  if i < 7:
    continue
  token = line.split()
  word_list.append(token[0])
f.close()

In [0]:
len(word_list)

7944

# Load Racial Names

In [0]:
# These are names in the paper
word_list = ['Alonzo', 'Jamel', 'Alphonse', 'Jerome', 'Leroy', 'Torrance',
       'Darnell', 'Lamar', 'Malik', 'Terrence', 'Adam', 'Harry', 'Josh',
       'Roger', 'Alan', 'Frank', 'Justin', 'Ryan', 'Andrew', 'Jack','Nichelle',
       'Shereen', 'Ebony', 'Latisha', 'Shaniqua', 'Jasmine', 'Tanisha',
       'Tia', 'Lakisha', 'Latoya', 'Amanda', 'Courtney', 'Heather',
       'Melanie', 'Katie', 'Betsy', 'Kristin', 'Nancy', 'Stephanie',
       'Ellen']

In [0]:
EEC['Person'].unique()

array(['Alonzo', 'Jamel', 'Alphonse', 'Jerome', 'Leroy', 'Torrance',
       'Darnell', 'Lamar', 'Malik', 'Terrence', 'Adam', 'Harry', 'Josh',
       'Roger', 'Alan', 'Frank', 'Justin', 'Ryan', 'Andrew', 'Jack', 'he',
       'this man', 'this boy', 'my brother', 'my son', 'my husband',
       'my boyfriend', 'my father', 'my uncle', 'my dad', 'Nichelle',
       'Shereen', 'Ebony', 'Latisha', 'Shaniqua', 'Jasmine', 'Tanisha',
       'Tia', 'Lakisha', 'Latoya', 'Amanda', 'Courtney', 'Heather',
       'Melanie', 'Katie', 'Betsy', 'Kristin', 'Nancy', 'Stephanie',
       'Ellen', 'she', 'this woman', 'this girl', 'my sister',
       'my daughter', 'my wife', 'my girlfriend', 'my mother', 'my aunt',
       'my mom', 'him', 'her'], dtype=object)

In [0]:
#For GloVe data
cn_embeddings = []
cn_embeddings.append([glove[word] for word in word_list if word in glove.vocab])
cn_embeddings = np.array(cn_embeddings[0])

In [0]:
#For word2vec data
cn_embeddings = []
cn_embeddings.append([word2vec[word] for word in word_list if word in word2vec.vocab])
cn_embeddings = np.array(cn_embeddings[0])

In [0]:
cn_embeddings.shape

(7099, 300)

In [0]:
#Sentence embeddings

In [0]:
import nltk

class Sentence:
    
    def __init__(self, sentence):
        self.raw = sentence
        normalized_sentence = sentence.replace("‘", "'").replace("’", "'")
        self.tokens = [t.lower() for t in nltk.word_tokenize(normalized_sentence)]

In [0]:
#Raw Sentence Embeddings
from sklearn.decomposition import TruncatedSVD, randomized_svd
from numpy.linalg import norm

def run_sif_benchmark(sentiment, sent1, sent2, model_str, freqs={}, a=0.001): 
    total_freq = sum(freqs.values())
    model = eval(model_str)
    embeddings = []
#    sen_list = EEC[EEC['Emotion']==sen]['Emotion word'].unique()
    #average sentiment embedding based on several emotional words of the same sentiment
#    sen_embeddings = np.average([model[item] for item in sen_list if item in model.vocab], axis=0)    
    tokens1 =  sent1.tokens
    tokens2 =  sent2.tokens
        
    tokens1 = [token for token in tokens1 if token in model and token.islower()]
    tokens2 = [token for token in tokens2 if token in model and token.islower()]
        
    weights1 = [a/(a+freqs.get(token,0)/total_freq) for token in tokens1]
    weights2 = [a/(a+freqs.get(token,0)/total_freq) for token in tokens2]
        
    embedding1 = np.array(np.average([model[token] for token in tokens1], axis=0, weights=weights1))
    embedding1 /= norm(embedding1)
    embedding2 = np.array(np.average([model[token] for token in tokens2], axis=0, weights=weights2))
    embedding2/= norm(embedding2)
    proj1 = embedding1.dot(model[sentiment])/norm(model[sentiment])
    proj2 = embedding2.dot(model[sentiment])/norm(model[sentiment])


    return proj1, proj2

In [0]:
#CN Sentence Embeddings
def run_conceptor_benchmark(sentiment,sent1, sent2, model_str, freqs={}, a=0.001, alpha = 1): 
    total_freq = sum(freqs.values())
    
    model = eval(model_str)
    embeddings = []
 #   sen_list = EEC[EEC['Emotion']==sen]['Emotion word'].unique()
 #   sen_embeddings = np.average([model[item] for item in sen_list if item in model.vocab], axis = 0) 
 #   print(sen_embeddings)
    # SIF requires us to first collect all sentence embeddings and then perform 
    # common component analysis.
        
    tokens1 =  sent1.tokens
    tokens2 =  sent2.tokens
        
    tokens1 = [token for token in tokens1 if token in model and token.islower()]
    tokens2 = [token for token in tokens2 if token in model and token.islower()]
        
    weights1 = [a/(a+freqs.get(token,0)/total_freq) for token in tokens1]
    weights2 = [a/(a+freqs.get(token,0)/total_freq) for token in tokens2]
        
    embedding1 = np.average([model[token] for token in tokens1], axis=0, weights=weights1)
    embedding2 = np.average([model[token] for token in tokens2], axis=0, weights=weights2)
        

    R = (cn_embeddings.T).dot(cn_embeddings) / cn_embeddings.shape[0]

    
    
    C = R @ np.linalg.inv (R + alpha ** (-1) * np.eye(300))# calculate the conceptor matrix
    
    negC = np.eye(300) - C
    #negC = np.eye(300) - SWC[model_str]
    
    
    embedding1 = negC.dot(embedding1.T).T
    embedding2 = negC.dot(embedding2.T).T
    embedding1 /= norm(embedding1)
    embedding2 /= norm(embedding2)
    proj1 = embedding1.dot(model[sentiment])/norm(model[sentiment])
    proj2 = embedding2.dot(model[sentiment])/norm(model[sentiment])
    return proj1, proj2

In [0]:
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_aa = sent_sent[(sent_sent['Race']=='African-American')]
  sent_ea = sent_sent[(sent_sent['Race']=='European') ]
 # print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_sif_benchmark(sen, sentence1, sentence2, 'word2vec', frequencies, 0.0001)
    score3, score4 = run_conceptor_benchmark(sen, sentence1, sentence2, 'word2vec', frequencies, 0.0001)
    aa.append(score1)
    ea.append(score2)
    aa_cn.append(score3)
    ea_cn.append(score4)
  p1 = scipy.stats.ttest_rel(aa, ea)
  #difference is significant, we can reject the null hypothesis that they are identical
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  print('gender difference on Raw and CN data:')
  print(abs((sum(aa)-sum(ea))/len(aa)), abs((sum(aa_cn)-sum(ea_cn))/len(aa_cn)))
  print('t value and p value :')
  print(p1,p2)


gender difference on Raw and CN data:
0.03544182361183609 0.034571777583579676
t value and p value :
Ttest_relResult(statistic=22.71370718654508, pvalue=5.7675480845435434e-86) Ttest_relResult(statistic=22.56741886042336, pvalue=3.912462492303365e-85)
gender difference on Raw and CN data:
0.03257069906326068 0.032970040426947254
t value and p value :
Ttest_relResult(statistic=18.214144652566763, pvalue=5.906807932032374e-61) Ttest_relResult(statistic=18.44383262232584, pvalue=3.4108038314288398e-62)
gender difference on Raw and CN data:
0.013294052843177398 0.011886645752740769
t value and p value :
Ttest_relResult(statistic=12.031849831865907, pvalue=1.948342991026626e-30) Ttest_relResult(statistic=10.893883409783168, pvalue=1.2375856003426063e-25)
gender difference on Raw and CN data:
0.030020961649561813 0.028432695710233866
t value and p value :
Ttest_relResult(statistic=25.484013113253344, pvalue=8.067141779016927e-102) Ttest_relResult(statistic=24.339017018983483, pvalue=3.009628

In [0]:
#Add extra names and pronouns
#load names and pronouns from EEC
name_pron = list(set(EEC['Person']))
for item in name_pron:
  num = item.rstrip().split(' ')
  if len(num)!=1:
    word_list.append(num[1])
  else:
    word_list.append(num[0])

In [0]:
len(word_list)

8068

In [0]:
cn_embeddings = []
cn_embeddings.append([word2vec[word] for word in word_list if word in word2vec.vocab])
cn_embeddings = np.array(cn_embeddings[0])

In [0]:
#For GloVe data
cn_embeddings = []
cn_embeddings.append([glove[word] for word in word_list if word in glove.vocab])
cn_embeddings = np.array(cn_embeddings[0])

In [0]:
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_aa = sent_sent[sent_sent['Race']=='African-American']
  sent_ea = sent_sent[sent_sent['Race']=='European']
 # print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_sif_benchmark(sen, sentence1, sentence2, 'glove', frequencies, 0.0001)
    score3, score4 = run_conceptor_benchmark(sen, sentence1, sentence2, 'glove', frequencies, 0.0001)
    aa.append(score1)
    ea.append(score2)
    aa_cn.append(score3)
    ea_cn.append(score4)
  p1 = scipy.stats.ttest_rel(aa, ea)
  #difference is significant, we can reject the null hypothesis that they are identical
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  print('gender difference on Raw and CN data:')
  print(abs((sum(aa)-sum(ea))/len(aa)), abs((sum(aa_cn)-sum(ea_cn))/len(aa_cn)))
  print('t value and p value :')
  print(p1,p2)


gender difference on Raw and CN data:
0.061123261630745884 0.04133149270875744
t value and p value :
Ttest_relResult(statistic=-43.67288118840131, pvalue=6.168340155161535e-202) Ttest_relResult(statistic=-38.07270716758232, pvalue=1.33652782615012e-172)
gender difference on Raw and CN data:
0.0858724936526182 0.05907615589288996
t value and p value :
Ttest_relResult(statistic=-57.158480759364984, pvalue=1.0926906450434775e-265) Ttest_relResult(statistic=-59.60153469226146, pvalue=3.1139904265216668e-276)
gender difference on Raw and CN data:
0.08717529532964859 0.07352710838274293
t value and p value :
Ttest_relResult(statistic=-43.47015139856975, pvalue=6.65376006275883e-201) Ttest_relResult(statistic=-43.870367423798236, pvalue=6.111179111001275e-203)
gender difference on Raw and CN data:
0.0326851227444638 0.028351460502684063
t value and p value :
Ttest_relResult(statistic=-25.323197119501536, pvalue=6.771104137212461e-101) Ttest_relResult(statistic=-22.518301515075287, pvalue=7.43